In [ ]:
from dunkerquesprint import Toflit, Portic

toflit = Toflit()
portic = Portic()

In [ ]:
import csv
# 1. définir le bureau de ferme et la direction des fermes de chacun des 6 ports
ports = ['Dunkerque', 'Calais', 'Boulogne sur Mer', 'Roscoff', 'Lorient', 'Bordeaux']

CSV_FILE_INPUT = '../../data/navigo_all_pointcalls_1787.csv'

done = {}

with open(CSV_FILE_INPUT, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for pointcall in reader:
        p = dict(pointcall)
        if p['pointcall'] in ports and p['pointcall'] not in done:
            print('- port : ' + p['pointcall'])
            done[p['pointcall']] = True
            bureau = p['ferme_direction']
            direction = p['ferme_bureau']
            print('- bureau de ferme : ' + bureau)    
            print('- direction de ferme : ' + direction)

In [ ]:
ports_info = [
    {
        "port": "Bordeaux",
        "bureau": "Bordeaux",
        "direction": "Bordeaux"
    },
    {
        "port": "Lorient",
        "bureau": "Lorient",
        "direction": "Lorient"
    },
    {
        "port": "Dunkerque",
        "bureau": "Dunkerque",
        "direction": "Dunkerque"
    },
    {
        "port": "Calais",
        "bureau": "Amiens",
        "direction": "Calais"
    },
    {
        "port": "Roscoff",
        "bureau": "Lorient",
        "direction": "Brest"
    },
    {
        "port": "Boulogne sur Mer",
        "bureau": "Amiens",
        "direction": "Boulogne-sur-Mer"
    },
    
]

In [ ]:
flows_50 = toflit.get_flows(year='1750')

In [ ]:
flows_87 = toflit.get_flows(year='1787')

In [ ]:
flows_89 = toflit.get_flows(year='1789')

In [ ]:
flows = toflit.get_flows()

In [ ]:
totaux = {
}

for flow in flows:
    year = flow['year']
    flow_type = flow['export_import']
    value = flow["value"]
    value = value.split(".")[0] if value != "" else "0"
    value = float(value) if value != "" else 0
    # cataloging directions
    if flow['customs_region'] != '':
        for port_group in ports_info:
            direction = port_group['direction']
            port = port_group["port"]
            if direction == flow['customs_region']:
                if port not in totaux:
                    totaux[port] = {
                        "direction": {},
                        "bureau": {}
                    }
                if year not in totaux[port]["direction"]:
                    totaux[port]["direction"][year] = {"Exports": 0, "Imports": 0}
                # add value
                totaux[port]["direction"][year][flow_type] += value
    # cataloging bureaux
    if flow['customs_office'] != '':
        for port_group in ports_info:
            bureau = port_group['bureau']
            port = port_group["port"]
            if bureau == flow['customs_office']:
                if port not in totaux:
                    totaux[port] = {
                        "direction": {},
                        "bureau": {}
                    }
                if year not in totaux[port]["bureau"]:
                    totaux[port]["bureau"][year] = {"Exports": 0, "Imports": 0}
                # add value
                totaux[port]["bureau"][year][flow_type] += value

In [ ]:
for port, categories in totaux.items():
    print("- données pour le port de " + port + " : ")
    for category, years in categories.items():
        print("  - niveau d'agrégation " + category + " : ")
        for year, types in years.items():
            print("    - " + str(year) + " : " + "exports : " + str(types['Exports']) + ", imports : " + str(types["Imports"]))

In [ ]:
series_directions = []
series_bureaux = []


for port, categories in totaux.items():
    years = categories['direction']
    for year, types in years.items():
        for type, value in types.items():
            series_directions.append({
                "port": port,
                "year": year,
                "type": type,
                "value": value
            })
for port, categories in totaux.items():
    years = categories['bureau']
    for year, types in years.items():
        for type, value in types.items():
            series_bureaux.append({
                "port": port,
                "year": year,
                "type": type,
                "value": value
            })

In [ ]:
from IPython.display import display
import pandas as pd


def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)
    
VegaLite({
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
    "mark": {"type": "line", "point": True},
    "title": "Imports et exports par direction attachée aux ports smoggleurs",
  "encoding": {
      "color": {"type": "nominal", "field": "type"},
        "y": {"type": "quantitative", "field": "value", "title": "valueur cumulée"},
        "x": {"type": "temporal", "field": "year", "title": "année"},
    "facet": {"type": "nominal", "field": "port", "title": "port concerné"}
  }, 
 "data": {
    "values": series_directions
  }
})

In [ ]:
VegaLite({
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "mark": {"type": "line", "point": True},
    "title": "Imports et exports par bureau attachée aux ports smoggleurs",
  "encoding": {
      "color": {"type": "nominal", "field": "type"},
        "y": {"type": "quantitative", "field": "value", "title": "valueur cumulée"},
        "x": {"type": "temporal", "field": "year", "title": "année"},
    "facet": {"type": "nominal", "field": "port", "title": "port concerné"}
  }, 
 "data": {
    "values": series_bureaux
  }
})